In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
from torchvision.models import vgg16
from HSI_class import HSI
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc
import zeroPadding

import time
start_time = time.time()

In [2]:
dataset_path = r"C:\Users\Asus TUF\Documents\code\TA\Hyperspectral oil spill detection datasets"

datasets = []

i = 0
for filename in os.listdir(dataset_path):
    if i>0:
        break
    file_path = os.path.join(dataset_path, filename)
    if os.path.isfile(file_path):  # Check if it's a file
        print(f"Processing file: {file_path}")
        hsi = HSI(file_path)
        datasets.append(hsi)
    i += 1

Processing file: C:\Users\Asus TUF\Documents\code\TA\Hyperspectral oil spill detection datasets\GM01.mat


In [3]:
def testWithDataset(n): 
    hsi_test = datasets[n]

    test_img = hsi_test.img
    test_gt = hsi_test.gt

    patch_size = 9
    half_patch = patch_size // 2

    height = test_img.shape[0]
    width = test_img.shape[1]

    matrix=zeroPadding.zeroPadding_3D(test_img,half_patch) #add 0 in every side of the data
    print(f"img shape: {test_img.shape}")
    print(f"img shape after padding {matrix.shape}")
    print(f"number of pixel {width * height}")

    print(f"ground truth shape: {test_gt.shape}")

    indices0 = np.argwhere(test_gt == 0)
    indices1 = np.argwhere(test_gt == 1)

    print(f"indices = 0 shape: {indices0.shape}")
    print(f"indices = 1 shape: {indices1.shape}")

    num_samples = 10

    random_indices0 = indices0[np.random.choice(len(indices0), num_samples, replace=False)]
    random_indices1 = indices1[np.random.choice(len(indices1), num_samples, replace=False)]

    test_indices = np.vstack((random_indices0, random_indices1))

    print(test_indices.shape)

    return test_indices, test_gt, matrix

In [4]:
def predict_batch(model, feature_extractor, batch_input, device):
    with torch.no_grad():  # Disable gradient computation
        test_features = feature_extractor(batch_input)  # shape: (10, 128)
        test_features_np = test_features.numpy()     # convert to NumPy

    predicted_classes = model.predict(test_features_np)
    probs = model.predict_proba(test_features_np)
    positive_probs = probs[:, 1]  # probability of class 1


    return predicted_classes, positive_probs


In [5]:
import joblib
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 64  # You can change this depending on your GPU capacity

model_path = r"C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\svm\svm_model.pkl"
model_name = model_path.split('\\')[-1]

print(f"Creating model {model_name}...")
svm_loaded = joblib.load(model_path)

print("Model loaded and moved to device")

Creating model svm_model.pkl...
Model loaded and moved to device


In [6]:
# import torch
# import torch.nn as nn
import torch.nn.functional as F

class ConvTo1D(nn.Module):
    def __init__(self):
        super(ConvTo1D, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=224, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))  # (batch_size, 128, 1, 1)
        self.flatten = nn.Flatten()               # (batch_size, 128)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.flatten(x)
        return x

feature_extractor = ConvTo1D()

In [7]:
test_indices, test_gt, matrix = testWithDataset(0)
print(test_gt.shape)
# for indice in test_indices:
#     print(test_gt[indice[0]][indice[1]])


img shape: (1243, 684, 224)
img shape after padding (1251, 692, 224)
number of pixel 850212
ground truth shape: (1243, 684)
indices = 0 shape: (820876, 2)
indices = 1 shape: (29336, 2)
(20, 2)
(1243, 684)


In [8]:
patch = 9
half_patch = patch // 2
input_patches = []
true_labels = []

# Prepare all patches
for x_pos, y_pos in test_indices:
    true_label = test_gt[x_pos][y_pos]

    selected_rows = matrix[x_pos:x_pos + 2*half_patch + 1, :]
    testing_patch = selected_rows[:, y_pos:y_pos + 2*half_patch + 1]

    patch_tensor = torch.tensor(testing_patch, dtype=torch.float32)
    patch_tensor = patch_tensor.unsqueeze(0).permute(0, 3, 1, 2)

    input_patches.append(patch_tensor)
    true_labels.append(true_label)

input_patches = torch.cat(input_patches, dim=0)  # Shape: (N, C, H, W)
true_labels = torch.tensor(true_labels)

with torch.no_grad():  # No gradient needed for feature extraction
    features = feature_extractor(input_patches)  # (100, 128)
    features_np = features.numpy() 

print(features_np.shape)

X = features_np
y = true_labels


predicted_classes = svm_loaded.predict(X)
probs = svm_loaded.predict_proba(X)
positive_probs = probs[:, 1]  # probability of class 1

# Combine both into a display
for i, (pred, prob) in enumerate(zip(predicted_classes, positive_probs)):
    print(f"Sample {i}: Predicted = {pred}, Prob(class 1) = {prob:.4f}")

# y_pred = svm.predict(X_val)
acc = accuracy_score(y, predicted_classes)
print(f"Validation Accuracy: {acc:.4f}")

(20, 128)
Sample 0: Predicted = 0, Prob(class 1) = 0.0000
Sample 1: Predicted = 0, Prob(class 1) = 0.0000
Sample 2: Predicted = 0, Prob(class 1) = 0.0000
Sample 3: Predicted = 0, Prob(class 1) = 0.0000
Sample 4: Predicted = 0, Prob(class 1) = 0.0000
Sample 5: Predicted = 0, Prob(class 1) = 0.0000
Sample 6: Predicted = 0, Prob(class 1) = 0.0000
Sample 7: Predicted = 0, Prob(class 1) = 0.0000
Sample 8: Predicted = 0, Prob(class 1) = 0.0000
Sample 9: Predicted = 0, Prob(class 1) = 0.0000
Sample 10: Predicted = 0, Prob(class 1) = 0.0000
Sample 11: Predicted = 0, Prob(class 1) = 0.0000
Sample 12: Predicted = 0, Prob(class 1) = 0.0000
Sample 13: Predicted = 0, Prob(class 1) = 0.0000
Sample 14: Predicted = 0, Prob(class 1) = 0.0000
Sample 15: Predicted = 0, Prob(class 1) = 0.0000
Sample 16: Predicted = 0, Prob(class 1) = 0.0000
Sample 17: Predicted = 0, Prob(class 1) = 0.0000
Sample 18: Predicted = 0, Prob(class 1) = 0.0000
Sample 19: Predicted = 0, Prob(class 1) = 0.0000


ValueError: Input arrays use different devices: cpu, cpu

In [ ]:


patch_size = 9
half_patch = patch_size // 2

scores = []

groundtruth = []
prediction = []
y_probs = []


for dataset in range(len(datasets)):
    print(f"tes: {dataset}")
    test_indices, test_gt, matrix = testWithDataset(dataset)

    total = len(test_indices)
    correct0 = 0
    correct1 = 0

    input_patches = []
    true_labels = []

    # Prepare all patches
    for x_pos, y_pos in test_indices:
        true_label = test_gt[x_pos][y_pos]

        selected_rows = matrix[x_pos:x_pos + 2*half_patch + 1, :]
        testing_patch = selected_rows[:, y_pos:y_pos + 2*half_patch + 1]

        patch_tensor = torch.tensor(testing_patch, dtype=torch.float32)
        patch_tensor = patch_tensor.unsqueeze(0).permute(0, 3, 1, 2)

        input_patches.append(patch_tensor)
        true_labels.append(true_label)

    input_patches = torch.cat(input_patches, dim=0)  # Shape: (N, C, H, W)
    true_labels = torch.tensor(true_labels)

    # Process in batches
    for i in tqdm(range(0, total, batch_size), desc="Predicting"):
        batch = input_patches[i:i+batch_size]
        labels = true_labels[i:i+batch_size]

        groundtruth.append(labels)

        preds, postive_class_probs = predict_batch(svm_loaded, feature_extractor, batch, device)

        prediction.append(preds)
        y_probs.append(postive_class_probs)

        for j in range(len(preds)):
            index = i + j
            print(f"{index+1}: prediction = {preds[j]}, expected: {labels[j].item()}")
            if preds[j] == labels[j].item():
                if labels[j].item() == 0:
                    correct0 += 1
                elif labels[j] == 1:
                    correct1 += 1

    correct = correct0 + correct1
    print(f"Score: {correct}/{total}")
    
    score = {
        'dataset': dataset,
        'correct_0': correct0,
        'correct_1': correct1,
        'correct_total': correct,
        'total': total
    }
    scores.append(score)
    # scores.append((f"dataset{dataset}", f'{correct0}/{total/2}', f'{correct1}/{total/2}', f'{correct}/{total}'))

In [ ]:
all_correct = 0
all_total = 0
all_correct0 = 0
all_correct1 = 0

for score in scores:
    dataset = score['dataset']
    correct0 = score['correct_0']
    correct1 = score['correct_1']
    correct = score['correct_total']
    total = score['total']
    print(f"dataset: {dataset}\t", f'{correct0}/{total/2}\t', f'{correct1}/{total/2}\t', f'{correct}/{total}\t')

    all_correct += correct
    all_total += total
    all_correct0 += correct0
    all_correct1 += correct1


print(f"total: \t\t {all_correct0}/{all_total/2} \t {all_correct1}/{all_total/2} \t {all_correct}/{all_total}")

print(f"acc: {all_correct/all_total}")

all_total_score = {
    'dataset': 'Total Dataset',
    'correct_0': all_correct0,
    'correct_1': all_correct1,
    'correct_total': all_correct,
    'total': all_total
}

scores.append(all_total_score)

In [ ]:
groundtruths = groundtruth
groundtruth = []

for x in groundtruths:
    for y in x:
        groundtruth.append(y)

predictions = prediction
prediction = []

for x in predictions:
    for y in x:
        prediction.append(y)


y_prob = []

for x in y_probs:
    for y in x:
        y_prob.append(y)

In [ ]:
print(len(groundtruth))
print(len(prediction))
print(len(y_prob))

In [ ]:
y_test = groundtruth
y_pred = prediction
correct = 0
total = 0

for x, y in zip(y_test, y_pred):
    total += 1
    if x == y:
        correct += 1

In [ ]:
print(f'{correct}/{total}')

In [ ]:
y_test_np = np.array([label.item() for label in y_test])
# Ensure labels are binary (0 and 1)
print("Unique values in y_test:", pd.Series(y_test_np).unique())

# Check if y_pred is probability (float) or hard prediction (int)
print("Sample y_pred values:", y_pred[:5])

In [ ]:


test_df = pd.DataFrame(
    {'True': y_test_np, 'Model': y_prob})

plt.figure(figsize=(7, 5))


fpr, tpr, _ = roc_curve(test_df['True'], test_df['Model'])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label=f'Model (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'r--', label='Random Guess')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Two Models')
plt.legend()
plt.show()

In [ ]:


y_true = np.array([int(label) for label in y_test_np])  # true labels
y_pred = prediction                          # predicted class labels (e.g., from predict_batch)

# Precision, Recall, F1
precision = precision_score(y_true, y_pred, average='macro')  # Use 'binary' if binary task
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

# Overall Accuracy (OA)
oa = accuracy_score(y_true, y_pred)

# Average Accuracy (AA) — mean of per-class accuracies
cm = confusion_matrix(y_true, y_pred)
per_class_acc = cm.diagonal() / cm.sum(axis=1)
aa = per_class_acc.mean()

# Print all metrics
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"OA:        {oa:.4f}")
print(f"AA:        {aa:.4f}")

In [ ]:

performance = {
    'AUC': float(roc_auc),
    'precision': float(precision),
    'recall': float(recall),
    'F1 Score': float(f1),
    'OA': float(oa),
    'AA': float(aa),
}
result_json = {
    'prediction' : scores,
    'performance' : performance,
}

In [ ]:
import json

print(result_json)

with open(f"performance/{model_name}_results.json", "w") as f:
    json.dump(result_json, f, indent=2)

print("JSON saved to results.json")

In [ ]:
end_time = time.time()
print(f"Run time: {end_time - start_time:.4f} seconds")